In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Dataset
from sklearn.datasets import load_digits

# PyTorch
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import training_logger

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

trainset = torchvision.datasets.MNIST(root='~/data',
                                        train=True,
                                        download=True,
                                        transform=transform)

testset = torchvision.datasets.MNIST(root='~/data',
                                        train=False,
                                        download=True,
                                        transform=transform)

In [3]:
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=2)

In [4]:
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=100,
                                         shuffle=False,
                                         num_workers=2)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # 28x28x32 -> 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3)  # 26x26x64 -> 24x24x64
        self.pool = nn.MaxPool2d(2, 2)  # 24x24x64 -> 12x12x64
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.dropout2 = nn.Dropout2d()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [6]:
torch.cuda.is_available()

True

In [7]:
device = torch.device("cuda:0")

In [8]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
epochs = 10
logger = training_logger.Train("b9d622b8-6ee9-46f1-a360-ea4fbf6cb8c2")
logger.create_train()
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[{epoch + 1}, {i+1}] loss: {running_loss / 100:.2}')
            running_loss = 0.0
    logger.send_metrics(epoch + 1, "loss", running_loss)
logger.end_train()

print('Finished Training')

Train created


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


[1, 100] loss: 2.2
[1, 200] loss: 1.6
[1, 300] loss: 0.83
[1, 400] loss: 0.62
[1, 500] loss: 0.53
[1, 600] loss: 0.48
Logs have added to database
[2, 100] loss: 0.43
[2, 200] loss: 0.42
[2, 300] loss: 0.39
[2, 400] loss: 0.36
[2, 500] loss: 0.35
[2, 600] loss: 0.33
Logs have added to database
[3, 100] loss: 0.32
[3, 200] loss: 0.3
[3, 300] loss: 0.3
[3, 400] loss: 0.27
[3, 500] loss: 0.26
[3, 600] loss: 0.27
Logs have added to database
[4, 100] loss: 0.25
[4, 200] loss: 0.26
[4, 300] loss: 0.24
[4, 400] loss: 0.22
[4, 500] loss: 0.23
[4, 600] loss: 0.23
Logs have added to database
[5, 100] loss: 0.21
[5, 200] loss: 0.21
[5, 300] loss: 0.2
[5, 400] loss: 0.2
[5, 500] loss: 0.21
[5, 600] loss: 0.19
Logs have added to database
[6, 100] loss: 0.19
[6, 200] loss: 0.19
[6, 300] loss: 0.19
[6, 400] loss: 0.18
[6, 500] loss: 0.16
[6, 600] loss: 0.17
Logs have added to database
[7, 100] loss: 0.16
[7, 200] loss: 0.16
[7, 300] loss: 0.16
[7, 400] loss: 0.16
[7, 500] loss: 0.18
[7, 600] loss: 0.1